In [8]:

!pip install opencv-python scikit-image

# Import libraries
import cv2
import numpy as np
from skimage.feature import hog
from skimage import filters
from tensorflow.keras.models import load_model



In [9]:
# Load the trained models
hog_model = load_model('hog_emotion_model.h5')
gabor_model = load_model('gabor_emotion_model.h5')
hog_gabor_model = load_model('hog_gabor_emotion_model.h5')



In [10]:
# Emotion labels
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
img_size = 48



In [11]:
# Feature extraction functions
def extract_hog_features(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    features, _ = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features



In [12]:
def extract_gabor_features(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    gabor_features = []
    for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]:
        gabor = filters.gabor(image, frequency=0.2, theta=theta)[1]
        gabor_features.append(gabor.flatten())
    return np.concatenate(gabor_features)

def extract_hog_gabor_features(image):
    hog_features = extract_hog_features(image)
    gabor_features = extract_gabor_features(image)
    return np.concatenate([hog_features, gabor_features])

# Face detection using Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')



In [13]:
# Function to process frame and predict emotion
def predict_emotion(frame, model_type='hog_gabor'):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Extract face ROI
        face_roi = gray[y:y+h, x:x+w]
        
        try:
            # Resize and preprocess
            face_roi = cv2.resize(face_roi, (img_size, img_size))
            face_roi = cv2.cvtColor(face_roi, cv2.COLOR_GRAY2BGR)  # Convert to 3-channel
            
            # Extract features based on model type
            if model_type == 'hog':
                features = extract_hog_features(face_roi)
                model = hog_model
            elif model_type == 'gabor':
                features = extract_gabor_features(face_roi)
                model = gabor_model
            else:  # hog_gabor
                features = extract_hog_gabor_features(face_roi)
                model = hog_gabor_model
            
            # Predict emotion
            features = np.expand_dims(features, axis=0)
            prediction = model.predict(features, verbose=0)
            emotion_idx = np.argmax(prediction)
            emotion = emotions[emotion_idx]
            confidence = np.max(prediction)
            
            # Display emotion and confidence
            cv2.putText(frame, f"{emotion} ({confidence:.2f})", (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
        except Exception as e:
            print(f"Error processing face: {e}")
    
    return frame



In [14]:
# Main webcam loop
def run_webcam(model_type='hog_gabor'):
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    print("Press 'q' to quit, 'h' for HOG, 'g' for Gabor, 'b' for HOG+Gabor")
    
    current_model = model_type
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Flip frame horizontally for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Process frame with current model
        processed_frame = predict_emotion(frame, current_model)
        
        # Display model type
        cv2.putText(processed_frame, f"Model: {current_model.upper()}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow('Emotion Detection', processed_frame)
        
        # Key controls
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('h'):
            current_model = 'hog'
        elif key == ord('g'):
            current_model = 'gabor'
        elif key == ord('b'):
            current_model = 'hog_gabor'
    
    cap.release()
    cv2.destroyAllWindows()

# Run the webcam with default model (HOG+Gabor)
run_webcam()

Press 'q' to quit, 'h' for HOG, 'g' for Gabor, 'b' for HOG+Gabor
